# Q1 profit 

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Construct credentials from service account key file
credentials = service_account.Credentials.from_service_account_file(
    'tche368-isom676-srvacct_srvacct.json')  # Update the file path as needed

# Construct a BigQuery client object
client = bigquery.Client(credentials=credentials)

# Revised and optimized query
QUERY = """
WITH product_filter AS (
    SELECT *
    FROM `machine_learning.products`
    WHERE prod_category NOT IN ("Gift Cards", "Other", "Front End Service", "Scanning Errors", "Customer Service-Misc", "Empties and Additionals")
),
valid_transactions AS (
    SELECT *
    FROM `machine_learning.transactions` a 
    join product_filter b on a.prod_id  = b.prod_id 
    WHERE trans_dt < "2020-03-01"
    AND a.prod_id IN (SELECT prod_id FROM product_filter)
        AND 
        -- Logic 1: Either sales_qty or sales_wgt is zero, but not both
        ((sales_qty = 0 AND sales_wgt <> 0) OR (sales_qty <> 0 AND sales_wgt = 0) or (sales_qty =1  AND sales_wgt <> 0))
        AND 
        -- Logics 2 and 3 are parallel conditions
        (
            (prod_category NOT IN ("Coupon", "returns") AND (sales_qty > 0 OR sales_wgt > 0))
            OR
            (prod_category IN ("Coupon", "returns") AND (sales_qty < 0 OR sales_wgt < 0))
        )
    AND sales_amt >= 0
),
transactions_per_day AS (
    SELECT cust_id, trans_dt, COUNT(DISTINCT trans_id) AS trans_per_day
    FROM valid_transactions
    GROUP BY cust_id, trans_dt
    HAVING trans_per_day <= 10
),
eligible_custs AS (
    SELECT v.cust_id
    FROM valid_transactions v
    JOIN transactions_per_day tpd ON v.cust_id = tpd.cust_id AND v.trans_dt = tpd.trans_dt
    GROUP BY v.cust_id
    HAVING COUNT(DISTINCT v.trans_id) >= 5
    AND COUNT(DISTINCT v.trans_dt) >= 5
    AND COUNT(v.trans_id) <= 20000
),
sampled_custs AS (
    SELECT cust_id
    FROM eligible_custs
    WHERE MOD(ABS(FARM_FINGERPRINT(CAST(cust_id AS STRING))), 1000) < 1
)
SELECT tx.*
FROM `valid_transactions` tx
JOIN sampled_custs ON tx.cust_id = sampled_custs.cust_id
WHERE tx.trans_dt < "2020-03-01"
"""

# Execute the query
query_job = client.query(QUERY)  # API request

# Convert to DataFrame
samp_df1 = query_job.to_dataframe()  # Waits for query to finish and converts it to DataFrame


In [ ]:
# Load product and profit margin data
product_profit_margin_df = pd.read_excel("C:/Users/ctlan/OneDrive/desktop/AI at Scale/HW/Product Category Profit Margin.xlsx")

# Merge with product_profit_margin to get profit margins
merged_df = pd.merge(sample_transaction, product_profit_margin_df, on='prod_category', how='left')

# Calculate profit for each transaction
merged_df['profit'] = merged_df['sales_amt'] * merged_df['profit_margin']

In [ ]:
product_profit_margin_df = pd.read_excel("C:/Users/ctlan/OneDrive/desktop/AI at Scale/HW/Product Category Profit Margin.xlsx")

# Merge with product_profit_margin to get profit margins
merged_df = pd.merge(sample_transaction, product_profit_margin_df, on='prod_category', how='left')

# Calculate profit for each transaction
merged_df['profit'] = merged_df['sales_amt'] * merged_df['profit_margin']

total_profit_by_customer = merged_df.groupby('cust_id')['profit'].sum().reset_index()

# Sort stores by total profit and select top 10
top_customer_by_profit = total_profit_by_customer.sort_values(by='profit', ascending=False).head(10)


top_customer_by_profit